# Датасет из телеграма

## Подготовка

Для начала вам понадобится скачать десктопную версию телеграма вот по этой [ссылке](https://tg-messenger.ru/?utm_source=direct&utm_medium=cpc&utm_campaign=telegram-for-desktop&utm_content=17238855481&utm_term=---autotargeting&ybaip=1&yclid=2328142818798206975).

После того, как вы удачно установили телеграм себе на компьютер, откройте его и выберите нужный канал / чат.

Далее:

* Найдите три точки в правом верхнем углу, нажмите и выберите опцию `Экспортировать чат` / `Export History`.
* Скачайте в формате `Json` текстовые данные себе на компьютер (уберите галочки
со всех вложений).

## Работаем с историей чата / канала

Для работы с данными нам понадобится `pandas`.

In [1]:
import pandas as pd

Загрузим файл с помощью метода `files.upload()`.

Скорость загрузки будет зависеть от скорости интернет-соединения и объема данных. Например, один из моих файлов на очень старом компьютере весом в 100 МБ загружался 7 мин.

В качестве примера я беру рандомный канал, на который подписан.

In [2]:
from google.colab import files
uploaded = files.upload()

Saving result.json to result.json


Посмотрим, скачался ли файл:

In [6]:
!ls

result.json  sample_data


Получилось! Откроем наш файл в пандасе.

In [3]:
data = pd.read_json("result.json")

Посмотрим на нашу таблицу.

In [4]:
data

,name,type,id,messages
0,Наука игры,public_channel,1917953520,"{'id': 1, 'type': 'service', 'date': '2023-09-..."
1,Наука игры,public_channel,1917953520,"{'id': 3, 'type': 'service', 'date': '2023-09-..."
2,Наука игры,public_channel,1917953520,"{'id': 6, 'type': 'service', 'date': '2023-09-..."
3,Наука игры,public_channel,1917953520,"{'id': 7, 'type': 'message', 'date': '2023-09-..."
4,Наука игры,public_channel,1917953520,"{'id': 8, 'type': 'message', 'date': '2023-09-..."
...,...,...,...,...
111,Наука игры,public_channel,1917953520,"{'id': 121, 'type': 'message', 'date': '2025-0..."
112,Наука игры,public_channel,1917953520,"{'id': 122, 'type': 'message', 'date': '2025-0..."
113,Наука игры,public_channel,1917953520,"{'id': 123, 'type': 'message', 'date': '2025-0..."
114,Наука игры,public_channel,1917953520,"{'id': 124, 'type': 'message', 'date': '2025-0..."


Из всей таблицы нас интересует последняя колонка `messages`. Там содержится информация о дате и тексте сообщения. Впечатляет! Посмотрим, как это выглядит.

In [9]:
data["messages"].iloc[10]

{'id': 14,
 'type': 'message',
 'date': '2023-09-11T14:21:07',
 'date_unixtime': '1694431267',
 'edited': '2023-12-19T19:19:58',
 'edited_unixtime': '1703002798',
 'from': 'Наука игры',
 'from_id': 'channel1917953520',
 'text': ['Не знаю, как вы, а я люблю слушать музыку:) Это удивительно лёгкий и приятный способ изучения языков - едешь себе в тесном автобусе или сидишь в офисе за скучной работой, а в наушниках в это время целый мир!\n\nДавайте мы с вами немного сыграем! Постом ниже я прикреплю 3 фрагмента разных песен и 3 опроса👇\n\nВаша задача: послушать каждый фрагмент, выбрать подходящий набор эмодзи и принять участие в опросах🎶\n\nP.S. Там будет играть музыка, поэтому, если вы находитесь в общественном месте, наденьте наушники😌\n',
  {'type': 'hashtag', 'text': '#мояигра'},
  ' ',
  {'type': 'hashtag', 'text': '#какзвучитроссия'},
  ''],
 'text_entities': [{'type': 'plain',
   'text': 'Не знаю, как вы, а я люблю слушать музыку:) Это удивительно лёгкий и приятный способ изучения яз

Раскроем содержимое колонки `messages` и создадим новую таблицу.

P.S. Дальше достаточно понимания на уровне «копировать-вставить». Для искушенных программистов напишу комментарии.

In [10]:
# раскрываем содержимое колонки messages
messages = pd.json_normalize(data["messages"])

# оставляем только сообщения с типом message
messages = messages[messages["type"] == "message"]

# напишем функцию, которая поможет корректно выгрузить сообщения

def get_text(row):
  parts = []
  if isinstance(row["text"], list):
    for item in row["text"]:
      if isinstance(item, dict) and "text" in item:
        parts.append(item["text"])
  elif pd.notnull(row["text"]):
    parts.append(str(row["text"]))
  if pd.notnull(row.get("caption", None)):
    parts.append(str(row["caption"]))
  return "\n".join(parts).strip()

# применим нашу функцию к таблице
messages["text"] = messages.apply(get_text, axis = 1)

# оставляем только те строки, где текст не пустой
messages = messages[messages["text"] != ""]

# оставляем только нужные колонки
result = messages[["date", "text"]]

Посмотрим на получившуюся таблицу.

In [11]:
result

,date,text
3,2023-09-04T16:26:12,Наука игры\n#welcomepost
4,2023-09-04T17:39:52,"Ещё как может! \nОнлайн-игра\nQuick, draw!\n#р..."
6,2023-09-09T08:52:50,сайт 1\nсайт 2\n#ресурс\n#мемы
7,2023-09-09T08:55:06,Ловите мои варианты мемов😄
10,2023-09-11T14:21:07,#мояигра\n#какзвучитроссия
...,...,...
111,2025-06-06T10:57:57,леший\nРусалка\nневиданных зверей\nдома\nигра ...
112,2025-07-01T13:15:37,игру БИНГО\nПочему люблю\nвиктории\n\n🖇Все мат...
113,2025-07-22T14:57:14,Второй сезон Чемпионата мира по русскому языку...
114,2025-08-05T09:05:40,"""Скажи по-русски!""\nв подборе синонимов\nкульт..."


Осталось взять текстовые данные и поработать с ними.

Сначала добавим все сообщения в пустой список `text`.

In [12]:
text = []
for i in result["text"]:
  text.append(i)

Превратим их в строку.

In [13]:
text = ''.join(text)

Выполним предобработку текста.

In [14]:
# приводим текст к нижнему регистру
text = text.lower()

# удаляем из него ненужные вещи (ссылки, хештеги, переносы, цифры)
import re
text = re.sub(r'https?://\S+|www\.\S+', '', text)
text = re.sub(r'#\S+', '', text)
text = re.sub(r'(?<=\w)-\s*(?=\w)', '', text)
text = re.sub('[^a-zа-я]+', ' ', text)

# скачиваем библиотеку для лемматизации
! pip install pymorphy3
import pymorphy3

# подключаем анализатор
from pymorphy3 import MorphAnalyzer
analyzer = MorphAnalyzer()

# в пустой массив «складываем» слова в начальной форме
lemmas = []
for i in text.split():
    lemmas.append(analyzer.parse(i)[0].normal_form)

# удаляем из них стоп-слова
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
nltk_stopwords = stopwords.words('russian')

lemmas_filtered = []

for i in lemmas:
  if i not in nltk_stopwords:
    lemmas_filtered.append(i)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 63.9 MB/s eta 0:00:00


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


На основе этого создадим частотный список слов.

In [15]:
from collections import Counter

counter = Counter(lemmas_filtered)
counter.most_common(30)

[('игра', 42),
 ('слово', 22),
 ('комментарий', 21),
 ('русский', 20),
 ('уровень', 20),
 ('это', 18),
 ('материал', 14),
 ('весь', 12),
 ('играть', 11),
 ('ребус', 11),
 ('язык', 11),
 ('новый', 11),
 ('иностранец', 11),
 ('всё', 11),
 ('первый', 10),
 ('подарок', 10),
 ('прикрепить', 10),
 ('alina', 9),
 ('galimullina', 9),
 ('rus', 9),
 ('занятие', 9),
 ('рки', 9),
 ('день', 9),
 ('тема', 9),
 ('идея', 9),
 ('мем', 8),
 ('ресурс', 8),
 ('студент', 8),
 ('картинка', 8),
 ('книга', 8)]

Ура! У нас получилось. А теперь попробуйте создать облако слов.

In [ ]:
# напишите код